# Descrição do problema
O problema de precificação é um problema de **regressão**, uma vez que os valores disponíveis serão utilizados para treinar um modelo para que ele possa estimar um valor numérico contínuo, o **preço**. 

Para o problema e a base de dados apresentados, alguns passos deverão ser seguidos para o sucesso da predição:
1. Remoção da coluna versão
2. Binarização das variáveis, usando a função `pandas.get_dummies()`
3. Aplicação dos datasets em 2 modelos diferentes: `sklearn.ensemble.RandomForestRegressor()` e `sklearn.neighbors.KNeighborsRegressor()`
4. Uso de 2 modelos de validação cruzada, para adequação de hiperparâmetros: `sklearn.model_selection.RandomizedSearchCV()` para o modelo RandomForest e `sklearn.model_selection.GridSearchCV()` para o KNeighbors 
5. Verificação de performance por MSE, usando a função `sklearn.metrics.mean_squared_error`
6. Comparação de performance entre as melhores versões de cada modelo
7. Aplicação do melhor dos dois modelos no dataset de teste
8. Predição da coluna preço para `cars_test.csv`
9. Exportar o arquivo `predicted.csv`


<b> Todas as escolhas serão devidamente justificadas ao longo do relatório </b>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
cars_train = pd.read_csv('datasets/cars_train.csv', engine='python',encoding='utf-16', sep=None)
cars_test = pd.read_csv('datasets/cars_test.csv',engine='python',encoding='utf-16',sep=None)

In [3]:
dummy_cols = ['marca','modelo','cambio','tipo','cor','tipo_vendedor','estado_vendedor','anunciante']

### Função para facilitar o tratamento dos dados

In [4]:
def clean_data(dataframe,dummy_cols=[]):
    """
    Limpa um dataframe para essa aplicação específica
    Args:
        dataframe (pandas.DataFrame): O dataframe que se quer limpar
        ev (float): o valor que se quer substituir no volume do motor de carros elétricos
        eletricos(list): lista dos três primeiros caracteres da versão do carro
        dummy_cols(list): lista das colunas que devem ser transformadas em valor numérico
    """
    #Copia o df para não alterar o df original
    df = dataframe.copy()
    binary=[]
    #Seleciona todas as colunas binárias
    for col in dataframe.columns:
        if (len(pd.unique(df[col]))<=2) and (df[col].isna().sum() !=0):
            binary.append(col)
    #Altera os valores das colunas para 1 ou 0
    for c in binary:
        df[c] = df[c].notnull().astype('int')
   #Faz a mesma operação para duas colunas que não entram no loop
    df['blindado'] = df['blindado'].map({'S':1,'N':0})
    df['elegivel_revisao'] = df['elegivel_revisao'].astype('int')
    df['num_fotos'] = df['num_fotos'].fillna(0) 
    #Altera colunas que não precisam ser dummy columns
    df['id'] = df['id'].astype('float')
    df['entrega_delivery'] = df['entrega_delivery'].astype('int')
    df['troca'] = df['troca'].astype('int')
    #Retira colunas desncessárias
    df.drop(['versao','cidade_vendedor'],axis=1,inplace=True)
    #Transforma colunas categóricas em colunas numéricas
    dummy_df = pd.get_dummies(df, columns=dummy_cols,drop_first=True)
    return dummy_df

### Alteração para valores numéricos
Os modelos utilizados não processam valores diferentes de valores numéricos

In [5]:
dummy_train = clean_data(cars_train,dummy_cols=dummy_cols)

In [6]:
dummy_test = clean_data(cars_test,dummy_cols=dummy_cols)

In [7]:
col_unseen = list(set(dummy_test.columns)-set(dummy_train.columns))

In [8]:
col_unused = list(set(dummy_train.columns)-set(dummy_test.columns))

In [9]:
y = dummy_train['preco']

In [10]:
dummy_train.drop(col_unused, axis=1, inplace=True)
dummy_test.drop(col_unseen,axis=1,inplace=True)

In [11]:
#Dividir em features e target
X = dummy_train.drop('id', axis=1)

In [12]:
seed=2710

## Utilização de 2 modelos
Serão comparados dois modelos diferentes para o problema posto, isso permite que haja comparação com pelo menos uma alternativa.
### Métrica de validação
Foi selecionado o erro quadrático médio (MSE) por ser uma boa métrica para modelos de regressão em que a varíavel alvo é contínua (um valor numérico arbitrário)
### RandomForestRegressor
Foi utilizada a validação cruzada `RandomizedSearchCV()` para seleção de hiperparâmetros da regressão de RandomForest porque há uma quantidade grande de combinações possíveis para os hiperparâmetros testados:
- max_depth
- max_features
- min_samples_split
- n_estimators

In [13]:
param_dist = {'max_depth':range(4,19,2),
             'max_features':range(2,21),
             'min_samples_split':range(2,11),
             'n_estimators':[100,200,300]
             }
rfr = RandomForestRegressor(random_state=seed)
mse = make_scorer(mean_squared_error)
random_search = RandomizedSearchCV(estimator=rfr,
                                  param_distributions=param_dist,
                                  n_iter=40,
                                  cv=5,
                                  scoring=mse)

In [14]:
rs = random_search.fit(X,y)

In [15]:
rs.best_score_

6008562426.728704

In [16]:
rs.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'max_features': 5,
 'max_depth': 4}

### RandomForestRegressor
Foi utilizada a validação cruzada `GridSearchCV()` para seleção de hiperparâmetros da regressão de KNeighbors porque há uma quantidade menor de combinações possíveis para os hiperparâmetros testados:
- n_neighbors
- weights

In [18]:
param_knn = {'n_neighbors': range(2,20),
            'weights':['uniform','distance']}
KNNReg = KNeighborsRegressor()
grid_knn = GridSearchCV(estimator=KNNReg,
                        param_grid=param_knn,
                        scoring=mse,
                        cv=5)

In [19]:
gs =grid_knn.fit(X, y)

In [20]:
gs.best_score_

8265696822.311914

In [21]:
gs.best_params_

{'n_neighbors': 2, 'weights': 'distance'}

## Modelo selecionado
Pela validação realizada acima, foi selecinado o modelo `RandomForestRegression()`, que apresentou o menor MSE entre os modelos testados

### Hiperparâmetros
Os hiperparrâmetros que melhor de adequam ao problema no modelo `RandomForestRegression()` são:
- max_depth = 4
- max_features = 5
- min_samples_split = 2
- n_estimators = 100

<b> Este será o modelo utilizado na predição dos preços do arquivo *cars_train.csv*</b>

In [22]:
X_test = dummy_test.drop('id', axis=1)

In [23]:
RandFR = RandomForestRegressor(max_depth=4, max_features=5, min_samples_split=2, n_estimators=100)

RandFR.fit(X,y)

RandomForestRegressor(max_depth=4, max_features=5)

In [24]:
y_predicitons = RandFR.predict(X_test)

In [26]:
predictions = pd.DataFrame({'id':cars_test['id'],'preco':y_predicitons})
predictions.to_csv("predicted.csv",index=False)